In [1]:
# Parameters
cpu = 8
group_name = "Astro-TE_NN"
mem_gb = 10


In [2]:
import pandas as pd
import subprocess
import numpy as np

In [3]:
#group_name = 'DG_Glut'

In [4]:
home_dir = '/home/qzeng_salk_edu/project/241018_pre_ml'
ct = group_name
dar_dir = '/data/combined_DARs_redo'
abc_dir = '/data/celltype_age_RPM_files'
#_type = 'up_peaks'

In [5]:
atac_rmp = pd.read_csv(f"{abc_dir}/{ct}_RPM.txt", sep = '\t')
# add three coolumns chr, start and end based on ethe index 
atac_rmp['chr'] = atac_rmp.index.str.split(':').str[0]
atac_rmp['start'] = atac_rmp.index.str.split(':').str[1].str.split('-').str[0].astype(int)
atac_rmp['end'] = atac_rmp.index.str.split(':').str[1].str.split('-').str[1].astype(int)
# put the chr start and end in the first column
cols = atac_rmp.columns.tolist()
cols = cols[-3:] + cols[:-3]
atac_rmp = atac_rmp[cols]
# remove chrY, chrM and chrX
atac_rmp = atac_rmp[~atac_rmp['chr'].isin(['chrY', 'chrM', 'chrX'])]
atac_rmp


,chr,start,end,Astro-TE NN:2mo,Astro-TE NN:9mo,Astro-TE NN:18mo
chr1:3399698-3400199,chr1,3399698,3400199,1.935104,1.425978,1.328092
chr1:3400380-3400881,chr1,3400380,3400881,0.408325,0.206821,0.426887
chr1:3671537-3672038,chr1,3671537,3672038,6.811330,7.347597,8.632596
chr1:3670856-3671357,chr1,3670856,3671357,2.272416,2.481855,2.739189
chr1:3672169-3672670,chr1,3672169,3672670,0.822567,1.055877,1.102790
...,...,...,...,...,...,...
chr9:124438702-124439203,chr9,124438702,124439203,0.520762,1.066762,0.972353
chr9:124441952-124442453,chr9,124441952,124442453,0.834403,1.219157,1.233228
chr9:124442507-124443008,chr9,124442507,124443008,0.491173,0.859941,0.889347
chr9:124456524-124457025,chr9,124456524,124457025,0.390571,0.587808,0.735194


In [6]:
# add rows in dar but not atac-rmp
dar_df = pd.read_csv(f'{home_dir}/ml_input/{ct}/{ct}.aDAR_gene.csv')
# make the index as chr:start-end
dar_df.index = dar_df['chr'] + ':' + dar_df['start'].astype(str) + '-' + dar_df['end'].astype(str)
dar_df['log2(18mo/2mo)'] = np.log2((dar_df['18mo'] + 1)/(dar_df['2mo'] + 1))

not_peak_dar =dar_df.loc[~dar_df.index.isin(atac_rmp.index)]
not_peak_dar = not_peak_dar[['chr','start','end','2mo','9mo','18mo']]
not_peak_dar.columns = atac_rmp.columns
atac_rmp = pd.concat([atac_rmp, not_peak_dar], axis = 0)
atac_rmp

,chr,start,end,Astro-TE NN:2mo,Astro-TE NN:9mo,Astro-TE NN:18mo
chr1:3399698-3400199,chr1,3399698,3400199,1.935104,1.425978,1.328092
chr1:3400380-3400881,chr1,3400380,3400881,0.408325,0.206821,0.426887
chr1:3671537-3672038,chr1,3671537,3672038,6.811330,7.347597,8.632596
chr1:3670856-3671357,chr1,3670856,3671357,2.272416,2.481855,2.739189
chr1:3672169-3672670,chr1,3672169,3672670,0.822567,1.055877,1.102790
...,...,...,...,...,...,...
chr12:26413792-26414293,chr12,26413792,26414293,13.291377,12.732988,12.059297
chr12:26413792-26414293,chr12,26413792,26414293,13.291377,12.732988,12.059297
chr4:107013326-107013827,chr4,107013326,107013827,10.916749,12.110266,8.057899
chrX:66649153-66649654,chrX,66649153,66649654,9.557166,8.555869,8.063414


In [7]:
atac_rmp.to_csv(f'{ct}.peak.bed', sep = '\t', index = False, header = False)

# use bedtools intersect to find the genes that overlap with the DMRs
command = f"bedtools intersect -a {ct}.peak.bed -b {home_dir}/mm10_gene_2kb.bed -wa -wb > {ct}.ATAC_peak_gene.bed"
subprocess.run(command, shell=True)

CompletedProcess(args='bedtools intersect -a Astro-TE_NN.peak.bed -b /home/qzeng_salk_edu/project/241018_pre_ml/mm10_gene_2kb.bed -wa -wb > Astro-TE_NN.ATAC_peak_gene.bed', returncode=0)

In [8]:
peak_gene_df = pd.read_csv(f'{ct}.ATAC_peak_gene.bed', sep='\t', header=None)
peak_gene_df.columns = ['chrom','peak_start','peak_end','2mo','9mo','18mo','gene_chrom','gene_start','gene_end','gene_id','strand', 'gene_name','gene_type']
peak_gene_df.index = peak_gene_df['chrom'] + ':' + peak_gene_df['peak_start'].astype(str) + '-' + peak_gene_df['peak_end'].astype(str)
# if the row index in dar and values in 
peak_gene_df

,chrom,peak_start,peak_end,2mo,9mo,18mo,gene_chrom,gene_start,gene_end,gene_id,strand,gene_name,gene_type
chr1:3399698-3400199,chr1,3399698,3400199,1.935104,1.425978,1.328092,chr1,3203901,3673498,ENSMUSG00000051951.5,-,Xkr4,protein_coding
chr1:3400380-3400881,chr1,3400380,3400881,0.408325,0.206821,0.426887,chr1,3203901,3673498,ENSMUSG00000051951.5,-,Xkr4,protein_coding
chr1:3671537-3672038,chr1,3671537,3672038,6.811330,7.347597,8.632596,chr1,3203901,3673498,ENSMUSG00000051951.5,-,Xkr4,protein_coding
chr1:3670856-3671357,chr1,3670856,3671357,2.272416,2.481855,2.739189,chr1,3203901,3673498,ENSMUSG00000051951.5,-,Xkr4,protein_coding
chr1:3672169-3672670,chr1,3672169,3672670,0.822567,1.055877,1.102790,chr1,3203901,3673498,ENSMUSG00000051951.5,-,Xkr4,protein_coding
...,...,...,...,...,...,...,...,...,...,...,...,...,...
chr12:26413792-26414293,chr12,26413792,26414293,13.291377,12.732988,12.059297,chr12,26412901,26417010,ENSMUSG00000098771.1,-,Mir6538,miRNA
chr12:26413792-26414293,chr12,26413792,26414293,13.291377,12.732988,12.059297,chr12,26304797,26417256,ENSMUSG00000020642.8,-,Rnf144a,protein_coding
chr4:107013326-107013827,chr4,107013326,107013827,10.916749,12.110266,8.057899,chr4,106908701,107051694,ENSMUSG00000061887.10,+,Ssbp3,protein_coding
chrX:66649153-66649654,chrX,66649153,66649654,9.557166,8.555869,8.063414,chrX,66647318,66663393,ENSMUSG00000036790.5,+,Slitrk2,protein_coding


In [9]:
peak_gene_df['DAR'] = peak_gene_df.index.map(dar_df['log2(18mo/2mo)'].to_dict())
# for values in DAR, if na == 0, if > 0 return 1, if < 0 return -1
peak_gene_df['DAR'] = peak_gene_df['DAR'].fillna(0)
peak_gene_df['DAR'] = np.sign(peak_gene_df['DAR'])
peak_gene_df.head()

,chrom,peak_start,peak_end,2mo,9mo,18mo,gene_chrom,gene_start,gene_end,gene_id,strand,gene_name,gene_type,DAR
chr1:3399698-3400199,chr1,3399698,3400199,1.935104,1.425978,1.328092,chr1,3203901,3673498,ENSMUSG00000051951.5,-,Xkr4,protein_coding,0.0
chr1:3400380-3400881,chr1,3400380,3400881,0.408325,0.206821,0.426887,chr1,3203901,3673498,ENSMUSG00000051951.5,-,Xkr4,protein_coding,0.0
chr1:3671537-3672038,chr1,3671537,3672038,6.811330,7.347597,8.632596,chr1,3203901,3673498,ENSMUSG00000051951.5,-,Xkr4,protein_coding,1.0
chr1:3670856-3671357,chr1,3670856,3671357,2.272416,2.481855,2.739189,chr1,3203901,3673498,ENSMUSG00000051951.5,-,Xkr4,protein_coding,0.0
chr1:3672169-3672670,chr1,3672169,3672670,0.822567,1.055877,1.102790,chr1,3203901,3673498,ENSMUSG00000051951.5,-,Xkr4,protein_coding,0.0


In [10]:
# drop dmr_id, gen_id, gene_strand, old-young
peak_gene_df['distance'] = np.minimum(abs(peak_gene_df['peak_start'] - peak_gene_df['gene_start']), abs(peak_gene_df['peak_start'] - peak_gene_df['gene_end']))
peak_gene_df = peak_gene_df.drop(columns=['chrom','peak_start','peak_end','gene_chrom','gene_id','strand','gene_type'])
peak_gene_df.head()

,2mo,9mo,18mo,gene_start,gene_end,gene_name,DAR,distance
chr1:3399698-3400199,1.935104,1.425978,1.328092,3203901,3673498,Xkr4,0.0,195797
chr1:3400380-3400881,0.408325,0.206821,0.426887,3203901,3673498,Xkr4,0.0,196479
chr1:3671537-3672038,6.811330,7.347597,8.632596,3203901,3673498,Xkr4,1.0,1961
chr1:3670856-3671357,2.272416,2.481855,2.739189,3203901,3673498,Xkr4,0.0,2642
chr1:3672169-3672670,0.822567,1.055877,1.102790,3203901,3673498,Xkr4,0.0,1329


In [11]:
peak_gene_df.to_csv(f"{home_dir}/ml_input/{ct}/{ct}.peak_gene.csv", index = False)

In [12]:
subprocess.run(f"rm {ct}.peak.bed", shell=True)
subprocess.run(f"rm {ct}.ATAC_peak_gene.bed", shell=True)

CompletedProcess(args='rm Astro-TE_NN.ATAC_peak_gene.bed', returncode=0)

## enhancer

In [13]:
atac_abc_dir = '/data/female-amb/ABC.Enhancer/ABC.By.ATAC.RPM.Raw.Cool'

In [14]:
mouse_genes = pd.read_csv(f"/home/qzeng_salk_edu/project/240321_abc_score/use.mouse_genes.csv", index_col =0)

In [15]:
def load_abc_df(ct, age):
    df = pd.read_csv(f'{atac_abc_dir}/{ct}/filtered.{ct}.{age}.abc_score.csv', index_col =0)
    if age == '8wk':
        age = '2mo'
    df.columns =  [f'{age}.activity',f'{age}.contact',f'{age}.abc_score']
    return df

In [16]:
young_abc_df = load_abc_df(ct, '8wk')
middle_abc_df = load_abc_df(ct, '9mo')
old_abc_df = load_abc_df(ct, '18mo')

In [17]:
# concat the three age groups
abc_df = pd.concat([young_abc_df, middle_abc_df, old_abc_df], axis=1)
#abc_df = abc_df.dropna()#
#abc_df = abc_df.fillna(0)

In [18]:
abc_df['chr'] = abc_df.index.map(lambda x: x.split(':')[0])
abc_df['start'] = abc_df.index.map(lambda x: int(x.split(':')[1].split('-')[0]))
abc_df['end'] = abc_df.index.map(lambda x: int(x.split(':')[1].split('-')[1]))
abc_df['gene_id'] = abc_df.index.map(lambda x: x.split(':')[1].split('-')[2])
abc_df['gene_name'] = abc_df['gene_id'].map(lambda x: mouse_genes.loc[x,'gene_name'])

abc_df['gene_start'] = abc_df['gene_id'].map(lambda x: mouse_genes.loc[x,'start'])
abc_df['gene_end'] = abc_df['gene_id'].map(lambda x: mouse_genes.loc[x,'end'])
# add eg_distance, based on the start is more close to gene_start ot gene_end; use the smaller absolute value
#abc_df drop chr, start, end, gene_start, gene_end
#abc_df = abc_df.drop(columns=['chr','start','end','gene_start','gene_end'])
abc_df


,2mo.activity,2mo.contact,2mo.abc_score,9mo.activity,9mo.contact,9mo.abc_score,18mo.activity,18mo.contact,18mo.abc_score,chr,start,end,gene_id,gene_name,gene_start,gene_end
chr1:3671537-3672038-ENSMUSG00000051951,6.811330,16.0,0.090792,7.347597,21.0,0.136107,8.632596,23.0,0.138905,chr1,3671537,3672038,ENSMUSG00000051951,Xkr4,3205901,3671498
chr1:3670856-3671357-ENSMUSG00000051951,2.272416,16.0,0.030290,2.481855,21.0,0.045974,2.739189,23.0,0.044076,chr1,3670856,3671357,ENSMUSG00000051951,Xkr4,3205901,3671498
chr1:4807551-4808052-ENSMUSG00000051951,33.476712,1.0,0.027889,31.262665,2.0,0.055153,29.680479,2.0,0.041529,chr1,4807551,4808052,ENSMUSG00000051951,Xkr4,3205901,3671498
chr1:7397752-7398253-ENSMUSG00000051951,71.918415,1.0,0.059915,64.582657,1.0,0.056968,68.847327,1.0,0.048165,chr1,7397752,7398253,ENSMUSG00000051951,Xkr4,3205901,3671498
chr1:5019243-5019744-ENSMUSG00000051951,41.998271,1.0,0.034989,37.793863,1.0,0.033338,41.170844,2.0,0.057606,chr1,5019243,5019744,ENSMUSG00000051951,Xkr4,3205901,3671498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chrX:166286666-166287167-ENSMUSG00000087159,NaN,NaN,NaN,NaN,NaN,NaN,18.640716,2.0,0.026070,chrX,166286666,166287167,ENSMUSG00000087159,Gm15246,169320622,169368190
chrX:166316590-166317091-ENSMUSG00000087159,NaN,NaN,NaN,NaN,NaN,NaN,22.968872,3.0,0.048184,chrX,166316590,166317091,ENSMUSG00000087159,Gm15246,169320622,169368190
chrX:169880224-169880725-ENSMUSG00000035299,NaN,NaN,NaN,NaN,NaN,NaN,2.110717,24.0,0.021191,chrX,169880224,169880725,ENSMUSG00000035299,Mid1,169685199,170005736
chrX:166510263-166510764-ENSMUSG00000035299,NaN,NaN,NaN,NaN,NaN,NaN,30.984854,2.0,0.025923,chrX,166510263,166510764,ENSMUSG00000035299,Mid1,169685199,170005736


In [19]:
abc_df.to_csv(f'{home_dir}/ml_input/{ct}/{ct}.abc_enhancer.peak_gene.csv', index=None)